**LEAD University - Minería de datos**

Python Project

**Contributors**
- Carolina Salas Moreno
- Deykel Bernard Salazar
- Esteban Ramirez Montano
- Kristhel Porras Mata
- Marla Gomez Hernández


## Requirements
**Step 1:** Please install Microsoft C++ Build Tools in your machine.

**Step 2:** Install Python 3.11.7

**Step 3:** Run the following code if this is your first time running it `pip install -r requirements.txt`

# Importar las librerías

In [10]:
# Main Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import umap.umap_ as umap

# Data Optimization
from sklearn_genetic import GASearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn_genetic.space import Integer, Categorical, Continuous
from xgboost import XGBRegressor
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor

#Feature Selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LassoCV 

# Clustering Libraries
from abc import ABCMeta, abstractmethod
from scipy.cluster.hierarchy import dendrogram, ward, single, complete, average, linkage, fcluster
from sklearn.cluster import KMeans
from pyclustering.cluster.kmedoids import kmedoids

# Dimensionality Reduction
from prince import PCA as PCA_Prince
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Evaluation Metrics
from sklearn.metrics import silhouette_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Regression Models
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Classification Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, RandomForestRegressor

# Additional Tools
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


## EDA

In [15]:
# Análisis Exploratorio de Datos (EDA)
class EDA:
    def __init__(self, file=None):
        """
        Inicializa la clase EDA y carga datos desde un archivo CSV si se proporciona.

        Parámetros:
            file (str): Ruta al archivo CSV. Si no se proporciona, se inicializa un DataFrame vacío.
        """
        self.__df = pd.read_csv(file) if file else pd.DataFrame()

    def head_df(self, n=5):
        return self.__df.head(n) if not self.__df.empty else "No se cargaron los datos :("

    def tail_df(self, n=5):
        return self.__df.tail(n) if not self.__df.empty else "No se cargaron los datos :("

    def check_data_types(self):
        return self.__df.dtypes

    def drop_irrelevant_columns(self, columns):
        self.__df.drop(columns=columns, inplace=True)

    def drop_missing_values(self):
        self.__df.dropna(inplace=True)

    def detect_outliers(self):
        num_df = self.__df.select_dtypes(include=['float64', 'int64'])
        if num_df.empty:
            return "No hay columnas numéricas en el DataFrame."

        Q1 = num_df.quantile(0.25)
        Q3 = num_df.quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((num_df < (Q1 - 1.5 * IQR)) | (num_df > (Q3 + 1.5 * IQR))).sum()
        Dicc_outliers = {col: outliers[col] for col in num_df.columns if outliers[col] > 0}

        return Dicc_outliers if Dicc_outliers else "No se detectaron valores atípicos en las columnas numéricas."

    def plot_scatter(self, col1, col2):
        plt.figure(figsize=(10, 6))
        sns.scatterplot(x=self.__df[col1], y=self.__df[col2])
        plt.title(f'Gráfico de Dispersión: {col1} vs {col2}')
        plt.xlabel(col1)
        plt.ylabel(col2)
        plt.grid()
        plt.show()

    def plot_histogram(self, col):
        plt.figure(figsize=(10, 6))
        sns.histplot(self.__df[col], kde=True)
        plt.title(f'Histograma de {col}')
        plt.xlabel(col)
        plt.ylabel('Frecuencia')
        plt.show()

    def plot_heatmap(self):
        num_df = self.__df.select_dtypes(include=['float64', 'int64'])
        if num_df.empty:
            return "No hay columnas numéricas para generar el mapa de calor."

        num_df = num_df.loc[:, num_df.apply(lambda x: np.std(x) > 0.01)]

        plt.figure(figsize=(12, 10))
        sns.heatmap(num_df.corr(), cmap="coolwarm", annot= True, linewidths=0.5, cbar=True) #annot=False es para que no se vean los numeros en los cuadros
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.title("Correlation heatmap", fontsize=18)
        plt.ion()

    def __str__(self):
        return f"Clase EDA - DataFrame de la forma: {self.__df.shape}"

    def get_df(self):
        """Devuelve una copia del df para que las familias de los algoritmos las utilicen"""
        return self.__df.copy()

# Data Optimization

In [20]:
class DataOptimization(EDA):
    def __init__(self, datos_eda):
        """
        Use the processed DataFrame from EDA to optimize models.

        Parameters:
        - datos_eda: This is the processed DataFrame from the EDA class.
        """
        self.__df = datos_eda.get_df()
        
        # Data components
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        
        # Models to evaluate
        self.models = {
            'LinearRegression': LinearRegression(),
            'DecisionTreeRegressor': DecisionTreeRegressor(),
            'RandomForestRegressor': RandomForestRegressor(),
            'Lasso': Lasso(),
            'Ridge': Ridge(),
            'KNeighborsRegressor': KNeighborsRegressor(),
            'XGBRegressor': XGBRegressor(random_state=42)
        }

        self.param_grids_genetic = self._get_param_grids_genetic()
        self.param_grids_exhaustive = self._get_param_grids_exhaustive()

#------------------------Data Split Components--------------------------------------------------------------

    def split_df(self, target_column, test_size=None, random_state=42):
        """
        Splits the dataframe into training and test sets.

        Parameters:
        - target_column: str -> Name of the target column (y).
        - test_size: float -> Proportion of the test set (if not provided, it is calculated from the entered percentage).
        - random_state: int -> Seed for randomization.

        Returns:
        - X_train, X_test, y_train, y_test: Split and preprocessed datasets.
        """
        while True:
            try:
                percent = float(input("Enter the percentage for the training set: (Example: 80) \n"))
                if 0 < percent < 100:
                    train_size = percent / 100
                    break # Exit the loop
                else:
                    print("The percentage must be between 1 and 99.")
            except ValueError:
                print("Invalid number. Try again.")

        while True:
            try:
                # Separate features (X) and target variable (y)
                X = self.__df.drop(columns=[target_column])
                y = self.__df[target_column]
                break  # Exit the loop if there are no errors
            except KeyError:
                print(f"The column '{target_column}' does not exist. Try again.")
                print("Available columns:")
                print(self.check_data_types())
                target_column = input("Enter the correct name of the target column: ")

        # Preprocess features (X), convert categorical variables to One-Hot Encoding
        import pandas as pd
        X = pd.get_dummies(X, drop_first=True)

        # Check if the target variable (y) is categorical and needs encoding
        if y.dtypes == 'object' or y.dtypes.name == 'category':
            from sklearn.preprocessing import LabelEncoder
            le = LabelEncoder()
            y = le.fit_transform(y)

        # Perform the split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size= 1 - train_size, random_state=random_state
        )

        print(f"Data split:\n- Training: {X_train.shape[0]} rows\n- Test: {X_test.shape[0]} rows")
        return X_train, X_test, y_train, y_test
    
#------------------------Genectic Search Components--------------------------------------------------------------

    def _get_param_grids_genetic(self):
        """Defines the parameters used for each algorithm in the genetic search."""
        return {
            'LinearRegression': {
                "clf__copy_X": Categorical([True, False]),
                "clf__fit_intercept": Categorical([True, False]),
                "clf__positive": Categorical([True, False])
            },
            'DecisionTreeRegressor': {
                "clf__max_depth": Integer(3, 10),
                'clf__min_samples_split': Integer(2, 10),
                'clf__min_samples_leaf': Integer(1, 4),
                'clf__random_state': Categorical([42])
            },
            'RandomForestRegressor': {
                "clf__n_estimators": Integer(50, 100),
                "clf__max_depth": Integer(5, 10),
                'clf__min_samples_split': Integer(2, 5),
                'clf__random_state': Categorical([42])
            },
            'Lasso': {
                'clf__alpha': Continuous(1.0, 1.0),
                'clf__fit_intercept': Categorical([True, False]),
                'clf__max_iter': Integer(1000, 2000),
                'clf__tol': Continuous(0.0001, 0.001),
                'clf__selection': Categorical(['cyclic', 'random'])
            },
            'Ridge': {
                'clf__alpha': Continuous(1.0, 1.0),
                'clf__fit_intercept': Categorical([True, False]),
                'clf__tol': Continuous(0.0001, 0.001),
                'clf__solver': Categorical(['auto', 'svd', 'cholesky'])
            },
            'KNeighborsRegressor': {
                'clf__n_neighbors': Integer(3, 7),
                'clf__weights': Categorical(['uniform', 'distance']),
                'clf__algorithm': Categorical(['auto', 'ball_tree', 'kd_tree'])
            },
            'XGBRegressor': {
                'clf__learning_rate': Continuous(0.01, 0.1),
                'clf__n_estimators': Integer(50, 100),
                'clf__max_depth': Integer(3, 5),
                'clf__subsample': Continuous(0.8, 1.0),
                'clf__colsample_bytree': Continuous(0.8, 1.0)
            }
        }

    def genetic_search(self):
        """
        Optimize regression models using genetic algorithms.
        """

        # Eliminar este if despues del testing ERM
        if self.X_train is None or self.X_test is None:
            print("Error: Debes ejecutar split_df() antes de llamar a genetic_search().")
            return

        results = {}
        
        # Feature selection
        lasso_cv = LassoCV(cv=5) 
        lasso_cv.fit(self.X_train, self.y_train)
        f_selection = SelectFromModel(lasso_cv)

        '''modelo_base_rfe = RandomForestRegressor(n_estimators=100, random_state=42)
        rfe = RFE(modelo_base_rfe, n_features_to_select=n_features)
        X_rfe_selected = rfe.fit_transform(self.X_selected, self.y)
        self.selected_features_rfe = self.X_selected.columns[rfe.support_]
        self.X_selected = self.X_selected[self.selected_features_rfe]
        print(f"Características finales seleccionadas por RFE: {list(self.selected_features_rfe)}")'''

        self.X_train = f_selection.transform(self.X_train)
        self.X_test = f_selection.transform(self.X_test)


        for name, model in self.models.items():
            pl = Pipeline([
              ('fs', f_selection), 
              ('clf', model), 
            ])            
            print(f"Entrenando {name} con método genético...")
            evolved_estimator = GASearchCV(
                estimator=pl,
                cv=5,
                scoring="neg_mean_squared_error",
                population_size=10,
                generations=5,
                tournament_size=3,
                elitism=True,
                crossover_probability=0.8,
                mutation_probability=0.1,
                param_grid=self.param_grids_genetic[name],
                algorithm="eaSimple",
                n_jobs=-1,
                error_score='raise',
                verbose=True
            )
            evolved_estimator.fit(self.X_train, self.y_train)
            results[name] = {
                'best_params': evolved_estimator.best_params_,
                'estimator': evolved_estimator.best_estimator_
            }
        return results

#------------------------Exhaustive Search Components--------------------------------------------------------------

    def _get_param_grids_exhaustive(self):
        """Defines the parameters used for each algorithm in the exhaustive search."""
        return {
            
            'LinearRegression': {
                "clf__copy_X": [True, False],
                "clf__fit_intercept": [True, False],
                "clf__positive": [True, False]
            },
            'DecisionTreeRegressor': {
                "clf__max_depth": [3, 5, 7, 10],
                'clf__min_samples_split': [2, 5, 10],
                'clf__min_samples_leaf': [1, 2, 4],
                'clf__random_state': [42]
            },
            'RandomForestRegressor': {
                "clf__n_estimators": [50, 100],
                "clf__max_depth": [5, 10],
                'clf__min_samples_split': [2, 5],
                'clf__random_state': [42]
            },
            'Lasso': {
                'clf__alpha': [1.0],
                'clf__fit_intercept': [True, False],
                'clf__max_iter': [1000, 2000],
                'clf__tol': [0.0001, 0.001],
                'clf__selection': ['cyclic', 'random']
            },
            'Ridge': {
                'clf__alpha': [1.0],
                'clf__fit_intercept': [True, False],
                'clf__tol': [0.0001, 0.001],
                'clf__solver': ['auto', 'svd', 'cholesky']
            },
            'KNeighborsRegressor': {
                'clf__n_neighbors': [3, 5, 7],
                'clf__weights': ['uniform', 'distance'],
                'clf__algorithm': ['auto', 'ball_tree', 'kd_tree']
            },
            'XGBRegressor': {
                'clf__learning_rate': [0.01, 0.1],
                'clf__n_estimators': [50, 100],
                'clf__max_depth': [3, 5],
                'clf__subsample': [0.8, 1.0],
                'clf__colsample_bytree': [0.8, 1.0]
            }
        }

    def exhaustive_search(self):
        """
        Solves optimization problems by creating a population or group of possible solutions to the problem.
        """
        results = {}
        lasso_cv = LassoCV(cv=5) 
        lasso_cv.fit(self.X_train, self.y_train)
        f_selection = SelectFromModel(lasso_cv)
        self.X_train = f_selection.transform(self.X_train)
        self.X_test = f_selection.transform(self.X_test)
        for name, model in self.models.items():
            
            pl = Pipeline([
              ('clf', model), 
            ])
            print(f"Entrenando {name} con método exhaustivo...")
            grid_search = GridSearchCV(
                estimator=pl,
                param_grid=self.param_grids_exhaustive[name],
                cv=5,
                scoring='neg_mean_squared_error',
                n_jobs=-1,
                verbose=1
            )
            grid_search.fit(self.X_train, self.y_train)
            results[name] = {
                'best_params': grid_search.best_params_,
                'estimator': grid_search.best_estimator_
            }
        return results
    
#------------------------Director Function--------------------------------------------------------------
    
    def opti_director(self, target_column, method='both', random_state=42):
        """
        This method orchestrates the optimization process for every model in this class.
        1. Make the data split
        2. Performs the optimization of models (genetic, exhaustive or both)
        3. Extract the best parameters in a clean format to use them in the models.
        
        Parameters:
        - target_column: str -> Name of the target column (y).
        - method: str -> What optimization method is going to be used (genetic, exhaustive or both)
        - test_size: float -> Percentage of the test set.
        - random_state: int -> Random seed for reproducibility.
        
        Returns:
        - dict -> Keeps the best parameters for each model.
        """
        #1. Make the data split
        self.X_train, self.X_test, self.y_train, self.y_test = self.split_df(
        target_column=target_column,
        random_state=random_state
        )
        
        # 2. Performs the optimization of models (genetic, exhaustive or both)
        best_params = {}
        
        if method.lower() == 'genetic' or method.lower() == 'both':
            print("\n=== Iniciando búsqueda genética ===")
            genetic_results = self.genetic_search()
            
            # 3. Extract the best parameters in a clean format to use them in the models.
            clean_genetic_params = {}
            for model_name, model_result in genetic_results.items():
                best_params_model = model_result['best_params']
                model_params = {param.replace('clf__', ''): value for param, value in best_params_model.items()}
                clean_genetic_params[model_name] = model_params
            
            best_params['genetic'] = clean_genetic_params
            
        if method.lower() == 'exhaustive' or method.lower() == 'both':
            print("\n=== Iniciando búsqueda exhaustiva ===")
            exhaustive_results = self.exhaustive_search()
            
            # 3. Extract the best parameters in a clean format to use them in the models.
            clean_exhaustive_params = {}
            for model_name, model_result in exhaustive_results.items():
                best_params_model = model_result['best_params']
                model_params = {param.replace('clf__', ''): value for param, value in best_params_model.items()}
                clean_exhaustive_params[model_name] = model_params
            
            best_params['exhaustive'] = clean_exhaustive_params
        
        return best_params

In [21]:
# MI TESTING
archivo_csv = "../dataset/potabilidad_V2.csv"
eda = EDA(file=archivo_csv)
print(eda.head_df())
optimizador = DataOptimization(eda)

best_params = optimizador.opti_director(
    target_column='Potability',
    method='genetic',             
)




   Unnamed: 0         ph    Hardness        Solids  Chloramines     Sulfate  \
0           4   8.316766  214.373394  22018.417441     8.059332  356.886136   
1           5   9.092223  181.101509  17978.986339     6.546600  310.135738   
2           6   5.584087  188.313324  28748.687739     7.544869  326.678363   
3           7  10.223862  248.071735  28749.716544     7.513408  393.663396   
4           8   8.635849  203.361523  13672.091764     4.563009  303.309771   

   Conductivity  Organic_carbon  Trihalomethanes  Turbidity Potability  
0    363.266516       18.436524       100.341674   4.628771         No  
1    398.410813       11.558279        31.997993   4.075075         No  
2    280.467916        8.399735        54.917862   2.559708         No  
3    283.651634       13.789695        84.603556   2.672989         No  
4    474.607645       12.363817        62.798309   4.401425         No  
Data split:
- Training: 1407 rows
- Test: 604 rows

=== Iniciando búsqueda genética ===

c:\Users\esteb\Documents\GitHub\MineriaAvanzada\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
c:\Users\esteb\Documents\GitHub\MineriaAvanzada\.venv\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


gen	nevals	fitness  	fitness_std	fitness_max	fitness_min
0  	10    	-0.239295	0.00213185 	-0.237549  	-0.24191   
1  	6     	-0.237987	0.00130592 	-0.237549  	-0.241905  
2  	7     	-0.237551	3.87778e-06	-0.237549  	-0.237559  
3  	7     	-0.23755 	2.90833e-06	-0.237549  	-0.237559  
4  	4     	-0.237549	2.77556e-17	-0.237549  	-0.237549  
5  	8     	-0.237549	2.77556e-17	-0.237549  	-0.237549  
Entrenando DecisionTreeRegressor con método genético...
gen	nevals	fitness  	fitness_std	fitness_max	fitness_min
0  	10    	-0.139649	0.0291812  	-0.106154  	-0.194121  
1  	6     	-0.106154	0          	-0.106154  	-0.106154  
2  	8     	-0.106154	0          	-0.106154  	-0.106154  
3  	8     	-0.106154	0          	-0.106154  	-0.106154  
4  	6     	-0.106154	0          	-0.106154  	-0.106154  
5  	7     	-0.106154	0          	-0.106154  	-0.106154  
Entrenando RandomForestRegressor con método genético...
gen	nevals	fitness  	fitness_std	fitness_max	fitness_min
0  	10    	-0.100836	0.028193   	

# Model

In [ ]:
class NoSupervisado(EDA):
    def __init__(self, datos_eda):
        # La clase ya utiliza el df procesado en la clase EDA
        df = datos_eda.get_df()
        super().__init__()
        self.__df = df

    @property
    def df(self):
        return self.__df

    @df.setter
    def df(self, p_df):
        self.__df = p_df

    def __byebye_object_values(self):
        # Elimina columnas de tipo 'object'
        self.__df = self.__df.select_dtypes(exclude=['object'])

    def calcular_metricas(self, labels):
        """
        Calcula métricas de evaluación para clustering.
        """
        data = self.__df.dropna()
        data = (data - data.mean()) / data.std()
        metrics = {
            "Índice de Silueta": silhouette_score(data, labels),
            "Calinski-Harabasz": calinski_harabasz_score(data, labels),
            "Davies-Bouldin": davies_bouldin_score(data, labels)
        }
        return metrics

    def kmeans(self, n_clusters):
        self.__byebye_object_values()
        data = self.__df
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        labels = kmeans.fit_predict(data)
        metrics = self.calcular_metricas(labels)
        print(f"Métricas para K-Means (n_clusters={n_clusters}): {metrics}")
        return metrics

    def k_medoids(self, n_clusters, metric='euclidean'):
        self.__byebye_object_values()
        data = self.__df
        
        # Convertir a numpy array si aún no lo es
        data_array = np.array(data)
        
        # Inicialización de medoides (seleccionar índices aleatorios)
        np.random.seed(42)  # Para reproducibilidad
        initial_medoids = np.random.choice(len(data_array), n_clusters, replace=False).tolist()
        
        # Crear y ejecutar el algoritmo KMedoids
        kmedoids_instance = kmedoids(data_array, initial_medoids)
        kmedoids_instance.process()
        
        # Obtener clusters y medoides
        clusters = kmedoids_instance.get_clusters()  # Lista de listas de índices
        medoids = kmedoids_instance.get_medoids()    # Lista de índices de medoides
        
        # Crear etiquetas en formato sklearn (un número para cada punto)
        labels = np.zeros(len(data_array), dtype=int)
        for cluster_idx, cluster in enumerate(clusters):
            for point_idx in cluster:
                labels[point_idx] = cluster_idx
        
        # Calcular métricas
        metrics = self.calcular_metricas(labels)
        print(f"Métricas para K-Medoids (n_clusters={n_clusters}, metric={metric}): {metrics}")
        
        return metrics

    def hac(self, n_clusters=3, method='ward'):
        self.__byebye_object_values()
        data = self.__df
        linkage_matrix = linkage(data, method=method)
        labels = fcluster(linkage_matrix, t=n_clusters, criterion='maxclust')
        metrics = self.calcular_metricas(labels)
        print(f"Métricas para HAC (n_clusters={n_clusters}, method={method}): {metrics}")
        return metrics

    def umap_model(self, n_components=2, n_neighbors=15):
        self.__byebye_object_values()
        data = self.__df
        modelo_umap = UMAP(n_components=n_components, n_neighbors=n_neighbors)
        components = modelo_umap.fit_transform(data)
        kmeans = KMeans(n_clusters=3, random_state=42)
        labels = kmeans.fit_predict(components)
        metrics = self.calcular_metricas(labels)
        print(f"Métricas para UMAP (n_components={n_components}, n_neighbors={n_neighbors}): {metrics}")
        return metrics

    def comparar_algoritmos(self, n_clusters):

        if self.__df.isnull().any().any():
          print("El DataFrame contiene valores nulos. Se eliminarán automáticamente para continuar.")
          self.__df.dropna(inplace=True)

        print("\nEjecutando K-Means...")
        kmeans_metrics = self.kmeans(n_clusters)

        print("\nEjecutando K-Medoids...")
        kmedoids_metrics = self.k_medoids(n_clusters)

        print("\nEjecutando HAC...")
        hac_metrics = self.hac(n_clusters=n_clusters)

        print("\nEjecutando UMAP...")
        umap_metrics = self.umap_model(n_components=2, n_neighbors=15)

        resultados = pd.DataFrame({
            "K-Means": kmeans_metrics,
            "K-Medoids": kmedoids_metrics,
            "HAC": hac_metrics,
            "UMAP": umap_metrics
        }).T

        print("\nComparación de Algoritmos:")
        print(resultados)
        return resultados

In [ ]:
class Supervisado(EDA):
    def __init__(self, datos_eda):
        # La clase ya utiliza el df procesado en la clase EDA
        df = datos_eda.get_df()
        super().__init__()
        self.__df = df

    @property
    def df(self):
        return self.__df

    @df.setter
    def df(self, p_df):
        self.__df = p_df

#-----------------Cosas del split----------------------------

    def obtener_datos_split(self):
        """
        Solicita al usuario el porcentaje de datos para entrenamiento.
        """
        while True:
            try:
                porcentaje = float(input("Introduce el porcentaje para el conjunto de entrenamiento: "))
                if 0 < porcentaje < 100:
                    return porcentaje / 100
                else:
                    print("El porcentaje debe estar entre 1 y 99.")
            except ValueError:
                print("Número no válido, use la mente.")

    def split_df(self, target_column, test_size=None, random_state=42):
      """
      Divide el dataframe en conjuntos de train y test.

      Parametros:
      - target_column: str -> Nombre de la columna objetivo (y).
      - test_size: float -> Proporcion del conjunto de prueba (si no se da un dato, se calcula del porcentaje ingresado).
      - random_state: int -> Semilla para la aleatorización.

      Returns:
      - X_train, X_test, y_train, y_test: Conjuntos divididos y preprocesados.
      """
      if test_size is None:
          # Si no se proporciona test_size, solicita al usuario el porcentaje
          test_size = 1 - self.obtener_datos_split()

      while True:
          try:
              # Separar las características (X) y la variable objetivo (y)
              X = self.__df.drop(columns=[target_column])
              y = self.__df[target_column]
              break  # Salir del bucle si no hay errores
          except KeyError:
              print(f"La columna '{target_column}' no existe. Intente nuevamente.")
              print("Columnas disponibles:")
              print(self.check_data_types())
              target_column = input("Ingrese el nombre correcto de la columna objetivo: ")

      # Preprocesar características (X), convertir variables categóricas a One-Hot Encoding
      import pandas as pd
      X = pd.get_dummies(X, drop_first=True)

      # Verificar si la variable objetivo (y) es categórica y necesita codificación
      if y.dtypes == 'object' or y.dtypes.name == 'category':
          from sklearn.preprocessing import LabelEncoder
          le = LabelEncoder()
          y = le.fit_transform(y)

      # Realizar el split
      X_train, X_test, y_train, y_test = train_test_split(
          X, y, test_size=test_size, random_state=random_state
      )

      print(f"Datos divididos:\n- Entrenamiento: {X_train.shape[0]} filas\n- Prueba: {X_test.shape[0]} filas")
      return X_train, X_test, y_train, y_test

#-----------------Evaluacion de modelos----------------------------
    def calcular_metricas(self, modelo, X_test, y_test, predicciones, modelo_nombre):
      """
      Calcula las métricas de evaluación del modelo y guarda los resultados en un diccionario.

      Parametros:
      - modelo: El modelo en uso
      - X_test: Datos de entrenamiento
      - y_test: Datos de prueba
      - predicciones: Predicciones del modelo
      - modelo_nombre: Nombre del modelo

      Returns:
      - resultados: Diccionario con métricas de evaluación.
      """
      mse = mean_squared_error(y_test, predicciones)
      r2 = r2_score(y_test, predicciones)
      mae = mean_absolute_error(y_test, predicciones)
      rmse = np.sqrt(mse)
      tolerancia = 0.1  # 10% de tolerancia
      precision_global = np.mean(np.abs(y_test - predicciones) <= (tolerancia * y_test)) * 100

      resultados = {
          'modelo': modelo_nombre,
          'MSE': mse,
          'R2': r2,
          'MAE': mae,
          'RMSE': rmse,
          'precision_global': precision_global,
          #'predicciones': predicciones.tolist(),
          #'valores_reales': y_test.tolist()
      }
      return resultados

    def calcular_metricas_clasificacion(self, modelo, X_test, y_test, predicciones, modelo_nombre):
      """
      Calcula las métricas de evaluación para modelos de clasificación y guarda los resultados en un diccionario.
      """
      from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

      accuracy = accuracy_score(y_test, predicciones)
      precision = precision_score(y_test, predicciones, average='weighted')
      recall = recall_score(y_test, predicciones, average='weighted')
      f1 = f1_score(y_test, predicciones, average='weighted')

      return {
          'modelo': modelo_nombre,
          'accuracy': accuracy,
          'precision': precision,
          'recall': recall,
          'f1_score': f1
      }

#------------------------Regression Models--------------------------------------------------------------

    def regre_lineal_simple(self, X_train, X_test, y_train, y_test):
      """
      Realiza una Regresión Lineal Simple y calcula múltiples métricas de rendimiento.
      Returns:
      - resultados: Diccionario con métricas de rendimiento del modelo
      """
      print("Iniciando Regresión Lineal Simple...")
      modelo = LinearRegression()
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)
      return self.calcular_metricas(modelo, X_test, y_test, predicciones, 'Regresión Lineal Simple')

    def regre_svm(self, X_train, X_test, y_train, y_test):
      """
      Realiza una Support Vector Machine y calcula múltiples métricas de rendimiento.
      Returns:
      - resultados: Diccionario con métricas de rendimiento del modelo
      """
      print("Iniciando Support Vector Machine (SVM)...")
      from sklearn.preprocessing import StandardScaler

      # Escalar los datos
      scaler = StandardScaler()
      X_train_scaled = scaler.fit_transform(X_train)
      X_test_scaled = scaler.transform(X_test)

      modelo = SVR(kernel='rbf', C=1.0, gamma='scale')
      modelo.fit(X_train_scaled, y_train)
      predicciones = modelo.predict(X_test_scaled)

      return self.calcular_metricas(modelo, X_test_scaled, y_test, predicciones, 'Support Vector Machine')


    def regre_regridge(self, X_train, X_test, y_train, y_test):
      """
      Realiza un Regresión Ridge y calcula múltiples métricas de rendimiento.
      Returns:
      - resultados: Diccionario con métricas de rendimiento del modelo
      """
      print("Iniciando Regresión Ridge...")
      modelo = Ridge(alpha = 1.0)
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)
      return self.calcular_metricas(modelo, X_test, y_test, predicciones, 'Regresión Ridge')

    def regre_decisionTree(self, X_train, X_test, y_train, y_test):
      """
      Realiza un Decision Tree Regressor y calcula múltiples métricas de rendimiento.
      Returns:
      - resultados: Diccionario con métricas de rendimiento del modelo
      """
      print("Iniciando DecisionTreeRegressor..")
      modelo = DecisionTreeRegressor(random_state=0)
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)
      return self.calcular_metricas(modelo, X_test, y_test, predicciones, 'Decision Tree Regressor')

    def regre_randomforest(self, X_train, X_test, y_train, y_test):
      """
      Realiza un Random Forest Regressor y calcula múltiples métricas de rendimiento.
      Returns:
      - resultados: Diccionario con métricas de rendimiento del modelo
      """
      print("Iniciando RandomForest Regressor..")
      modelo = RandomForestRegressor(max_depth=2, random_state=0)
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)
      return self.calcular_metricas(modelo, X_test, y_test, predicciones, 'Random Forest Regressor')

    def regre_gradient_boosting(self, X_train, X_test, y_train, y_test):
      """
      Realiza un Grandient Boostsing Regressor y calcula múltiples métricas de rendimiento.
      Returns:
      - resultados: Diccionario con métricas de rendimiento del modelo
      """
      print("Iniciando Grandient Boostsing Regressor..")
      modelo = GradientBoostingRegressor(random_state=0)
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)
      return self.calcular_metricas(modelo, X_test, y_test, predicciones, 'Grandient Boostsing Regressor')

    def regre_xgboost(self, X_train, X_test, y_train, y_test):
      """
      Realiza un XGBoost Regressor y calcula múltiples métricas de rendimiento.
      Returns:
      - resultados: Diccionario con métricas de rendimiento del modelo
      """
      print("Iniciando XGBoost Regressor..")
      modelo = GradientBoostingRegressor(random_state=0)
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)
      return self.calcular_metricas(modelo, X_test, y_test, predicciones, 'XGBoost Regressor')

#------------------------Classification Models--------------------------------------------------------------

    def classi_decision_tree(self, X_train, X_test, y_train, y_test):
      """
      Realiza un modelo de clasificación usando Árbol de Decisión y calcula métricas de rendimiento.
      """
      print("Iniciando Decision Tree Classifier...")
      from sklearn.tree import DecisionTreeClassifier

      modelo = DecisionTreeClassifier(random_state=0)
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)

      return self.calcular_metricas_clasificacion(modelo, X_test, y_test, predicciones, 'Decision Tree')

    def classi_knn(self, X_train, X_test, y_train, y_test):
      """
      Realiza un modelo de clasificación usando K-Nearest Neighbors y calcula métricas de rendimiento.
      """
      print("Iniciando K-Nearest Neighbors Classifier...")
      from sklearn.neighbors import KNeighborsClassifier

      modelo = KNeighborsClassifier()
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)

      return self.calcular_metricas_clasificacion(modelo, X_test, y_test, predicciones, 'K-Nearest Neighbors')

    def classi_random_forest(self, X_train, X_test, y_train, y_test):
      """
      Realiza un modelo de clasificación usando Random Forest y calcula métricas de rendimiento.
      """
      print("Iniciando Random Forest Classifier...")
      from sklearn.ensemble import RandomForestClassifier

      modelo = RandomForestClassifier(random_state=0)
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)

      return self.calcular_metricas_clasificacion(modelo, X_test, y_test, predicciones, 'Random Forest')

    def classi_adaboost(self, X_train, X_test, y_train, y_test):
      """
      Realiza un modelo de clasificación usando AdaBoost y calcula métricas de rendimiento.
      """
      print("Iniciando AdaBoost Classifier...")
      from sklearn.ensemble import AdaBoostClassifier

      modelo = AdaBoostClassifier(random_state=0)
      modelo.fit(X_train, y_train)
      predicciones = modelo.predict(X_test)

      return self.calcular_metricas_clasificacion(modelo, X_test, y_test, predicciones, 'AdaBoost')

In [ ]:
class Start:
    def __init__(self):
        self.eda = None
        self.supervisado = None
        self.no_supervisado = None
        self.split_data = None


    def mostrar_menu(self):
        while True:
            print("\n--- Menú Principal ---")
            print("1. 📁 Carga de datos en formato CSV y completar EDA")
            print("2. 🪐 Ejecutar modelo")
            print("3. 🛑 Salir")
            opcion = input("Seleccione una opción: ")

            if opcion == "1":
                self.datos_eda()
            elif opcion == "2":
                self.models_menu()
            elif opcion == "3":
                print("Saliendo del programa...")
                break
            else:
                print("Opción no válida. Intente de nuevo.")

    def models_menu(self):
      while True:
        print("\n--- ¿Qué problema necesita resolver? ---")
        print("1. 🔍 Clasificación: Asigne etiquetas a sus datos")
        print("2. 📈 Regresión: Prediga valores continuos")
        print("3. 🧩 Aprendizaje No Supervisado: Descubra patrones ocultos")
        print("4. 🛑 Volver al menú principal")
        opcion = input("Seleccione una opción: ")

        # Resetear split_data antes de cambiar de modelo
        self.split_data = None
        self.supervisado = None

        if opcion == "1":
            self.classi_modelos()
        elif opcion == "2":
            self.regre_modelos()
        elif opcion == "3":
                if self.eda and not self.eda.get_df().empty:
                  self.no_supervisado = NoSupervisado(self.eda)
                  n_clusters = int(input("Ingrese el número de clusters: "))
                  self.no_supervisado.comparar_algoritmos(n_clusters=n_clusters)
                else:
                    print("Primero cargue los datos para poder realizar aprendizaje no supervisado.")
        elif opcion == "4":
            print("Saliendo del programa...")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

    def datos_eda(self):
        while True:
            print("\n ----EDA----")
            print("1. 📂 Carga de datos")
            print("2. 🔍 Mostrar head del DataFrame")
            print("3. 📊 Revisar los tipos de datos")
            print("4. ✂️ Eliminar columnas")
            print("5. 🧹 Eliminar valores NULOS")
            print("6. ⚠️ Detectar valores atipicos")
            print("7. 📈 Graficar relación entre dos variables")
            print("8. 📉 Graficar histograma")
            print("9. 🌡 HeatMap: Generar mapa de calor")
            print("0. 🛑 Volver al menú principal")
            opcion = input("Seleccione una opción: ")

            if opcion == "1":
                my_data = input("¿Cómo se llama el CSV? ")
                try:
                    self.eda = EDA(file=my_data)
                    print("Instancia de EDA creada y datos cargados exitosamente.")
                except Exception as e:
                    print(f"Error al cargar los datos: {e}")
            elif opcion == "2":
                if self.eda:
                    print(self.eda.head_df())
                else:
                    print("Primero cargue los datos.")
            elif opcion == "3":
                if self.eda:
                    print(self.eda.check_data_types())
                else:
                    print("Primero cargue los datos.")
            elif opcion == "4":
                if self.eda:
                    columnas = input("Ingrese los nombres de las columnas a eliminar, separadas por comas: ").split(',')
                    columnas = [col.strip() for col in columnas]
                    try:
                        self.eda.drop_irrelevant_columns(columnas)
                        print(f"Columnas eliminadas: {', '.join(columnas)}")
                    except Exception as e:
                        print(f"Error al eliminar columnas: {e}")
                else:
                    print("Primero cargue los datos.")
            elif opcion == "5":
                if self.eda:
                    self.eda.drop_missing_values()
                    print("Valores nulos eliminados.")
                else:
                    print("Primero cargue los datos.")
            elif opcion == "6":
                if self.eda:
                    print(self.eda.detect_outliers())
                else:
                    print("Primero cargue los datos.")
            elif opcion == "7":
                if self.eda:
                    print("\n ***Variables disponibles***")
                    print(self.eda.check_data_types())
                    col1 = input("Ingrese el nombre de la primera variable: ")
                    col2 = input("Ingrese el nombre de la segunda variable: ")
                    try:
                        self.eda.plot_scatter(col1, col2)
                    except Exception as e:
                        print(f"Error al graficar: {e}")
                        break
                else:
                    print("Primero cargue los datos.")
            elif opcion == "8":
                if self.eda:
                    print("\n ***Variables disponibles***")
                    print(self.eda.check_data_types())
                    histogram_col = input("Ingrese el nombre de una variable a graficar: ")
                    try:
                        self.eda.plot_histogram(histogram_col)
                    except Exception as e:
                        print(f"Error al graficar: {e}")
                else:
                    print("Primero cargue los datos.")
            elif opcion == "9":
                if self.eda:
                    self.eda.plot_heatmap()
                    print("El programa se detendrá después de mostrar el gráfico.")
                    exit()
                else:
                    print("Primero cargue los datos.")
            elif opcion == "0":
                break
            else:
                print("Opción no válida. Intente de nuevo.")

    def regre_modelos(self):
      if self.supervisado is None:
          if self.eda:
              self.supervisado = Supervisado(self.eda)
          else:
              print("Primero debe cargar los datos")
              return

      if self.split_data is None:
          print("\n ***Variables disponibles***")
          print(self.eda.check_data_types())
          target_column = input("\n Ingrese el nombre de la columna objetivo: ")
          self.split_data = self.supervisado.split_df(target_column)

      X_train, X_test, y_train, y_test = self.split_data

      modelos = [
          self.supervisado.regre_lineal_simple,
          self.supervisado.regre_svm,
          self.supervisado.regre_regridge,
          self.supervisado.regre_decisionTree,
          self.supervisado.regre_randomforest,
          self.supervisado.regre_gradient_boosting,
          self.supervisado.regre_xgboost
      ]

      resultados = []
      for modelo in modelos:
          resultados.append(modelo(X_train, X_test, y_train, y_test))

      print("\n--- Resultados del Benchmarking ---")
      for resultado in resultados:
          print(f"{resultado['modelo']}: R2={resultado['R2']:.4f}, RMSE={resultado['RMSE']:.4f}, MAE={resultado['MAE']:.4f}")

      # Opción de graficar resultados
      graficar = input("\n¿Desea graficar los resultados? (S/N): ").strip().upper()
      if graficar == 'S':
        # Preparar datos para la gráfica
        nombres_modelos = [resultado['modelo'] for resultado in resultados]
        r2_scores = [resultado['R2'] for resultado in resultados]
        rmse_scores = [resultado['RMSE'] for resultado in resultados]
        mae_scores = [resultado['MAE'] for resultado in resultados]

        # Crear la gráfica de barras comparativa
        plt.figure(figsize=(15, 6))

        # Gráfica de R2
        plt.subplot(1, 3, 1)
        bars1 = plt.bar(nombres_modelos, r2_scores)
        plt.title('R2 Scores')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('R2')
        # Añadir valores en las barras
        for bar in bars1:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                     f'{height:.4f}',
                     ha='center', va='bottom', rotation=0)

        # Gráfica de RMSE
        plt.subplot(1, 3, 2)
        bars2 = plt.bar(nombres_modelos, rmse_scores)
        plt.title('Root Mean Squared Error')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('RMSE')
        # Añadir valores en las barras
        for bar in bars2:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                     f'{height:.4f}',
                     ha='center', va='bottom', rotation=0)

        # Gráfica de MAE
        plt.subplot(1, 3, 3)
        bars3 = plt.bar(nombres_modelos, mae_scores)
        plt.title('Mean Absolute Error')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('MAE')
        # Añadir valores en las barras
        for bar in bars3:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                     f'{height:.4f}',
                     ha='center', va='bottom', rotation=0)

        plt.tight_layout()
        plt.show()
        exit()

    def classi_modelos(self):
        if graficar == 'S':
          # Preparar datos para la gráfica
          nombres_modelos = [resultado['modelo'] for resultado in resultados]
          accuracy_scores = [resultado['accuracy'] for resultado in resultados]
          precision_scores = [resultado['precision'] for resultado in resultados]
          recall_scores = [resultado['recall'] for resultado in resultados]
          f1_scores = [resultado['f1_score'] for resultado in resultados]

          # Crear la gráfica de barras comparativa
          plt.figure(figsize=(15, 6))

          # Gráfica de Accuracy
          plt.subplot(1, 4, 1)
          bars1 = plt.bar(nombres_modelos, accuracy_scores)
          plt.title('Accuracy')
          plt.xticks(rotation=45, ha='right')
          plt.ylabel('Accuracy')
          # Añadir valores en las barras
          for bar in bars1:
              height = bar.get_height()
              plt.text(bar.get_x() + bar.get_width()/2., height,
                      f'{height:.4f}',
                      ha='center', va='bottom', rotation=0)

          # Gráfica de Precision
          plt.subplot(1, 4, 2)
          bars2 = plt.bar(nombres_modelos, precision_scores)
          plt.title('Precision')
          plt.xticks(rotation=45, ha='right')
          plt.ylabel('Precision')
          # Añadir valores en las barras
          for bar in bars2:
              height = bar.get_height()
              plt.text(bar.get_x() + bar.get_width()/2., height,
                      f'{height:.4f}',
                      ha='center', va='bottom', rotation=0)

          # Gráfica de Recall
          plt.subplot(1, 4, 3)
          bars3 = plt.bar(nombres_modelos, recall_scores)
          plt.title('Recall')
          plt.xticks(rotation=45, ha='right')
          plt.ylabel('Recall')
          # Añadir valores en las barras
          for bar in bars3:
              height = bar.get_height()
              plt.text(bar.get_x() + bar.get_width()/2., height,
                      f'{height:.4f}',
                      ha='center', va='bottom', rotation=0)

          # Gráfica de F1-Score
          plt.subplot(1, 4, 4)
          bars4 = plt.bar(nombres_modelos, f1_scores)
          plt.title('F1-Score')
          plt.xticks(rotation=45, ha='right')
          plt.ylabel('F1-Score')
          # Añadir valores en las barras
          for bar in bars4:
              height = bar.get_height()
              plt.text(bar.get_x() + bar.get_width()/2., height,
                      f'{height:.4f}',
                      ha='center', va='bottom', rotation=0)

          plt.tight_layout()
          plt.show()
          exit()

Ejecucion del programa con estructura pythonica

In [ ]:
# Ejecución del menú principal
if __name__ == "__main__":
    start = Start()
    start.mostrar_menu()